In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("train.csv")
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


# 결측치 예측을 위한 데이터 전처리

## 필요없어 보이는 feature 삭제
PassengerId, Name 삭제

In [4]:
train.drop(columns=["PassengerId","Name"], inplace=True)
train

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


## Cabin 나눠서 표현하기
Cabin을 확인해보면 총 3부분으로 이루어져 있으므로 이를 분석하기 쉽게 3부분으로 쪼개어 표현해 놓는다.

In [5]:
train[["Cabin1","Cabin2", "Cabin3"]] = train["Cabin"].str.split("/", expand=True)
train
# 새롭게 추가된 Cabin1, Cabin2, Cabin3 feature들을 확인할 수 있다.

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0,P
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0,S
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0,S
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0,S
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,A,98,P
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,G,1499,S
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,G,1500,S
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,E,608,S


In [6]:
# Cabin은 삭제해준다.
train.drop(columns="Cabin", inplace=True)
train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0,P
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0,S
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,A,0,S
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,A,0,S
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,A,98,P
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False,G,1499,S
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,G,1500,S
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,E,608,S


## 범주형 label encoding하기
HomePlanet, CryoSleep, Destination, VIP, Cabin1, Cabin3 범주형들을 인코딩

In [7]:
from sklearn.preprocessing import LabelEncoder

le_HomePlanet = LabelEncoder()
train["HomePlanet"] = le_HomePlanet.fit_transform(train["HomePlanet"])

le_CryoSleep = LabelEncoder()
train["CryoSleep"] = le_CryoSleep.fit_transform(train["CryoSleep"])

le_Destination = LabelEncoder()
train["Destination"] = le_Destination.fit_transform(train["Destination"])

le_VIP = LabelEncoder()
train["VIP"] = le_VIP.fit_transform(train["VIP"])

le_Cabin1 = LabelEncoder()
train["Cabin1"] = le_Cabin1.fit_transform(train["Cabin1"])

le_Cabin3 = LabelEncoder()
train["Cabin3"] = le_Cabin3.fit_transform(train["Cabin3"])

In [8]:
train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False,1,0,0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,True,5,0,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,0,0,1
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,0,0,1
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,True,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,1,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,False,0,98,0
8689,0,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,False,6,1499,1
8690,0,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,True,6,1500,1
8691,1,0,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,False,4,608,1


## 결측치 데이터 뽑기(범주형만)

In [9]:
le_HomePlanet.classes_

array(['Earth', 'Europa', 'Mars', nan], dtype=object)

In [35]:
HomePlanet_Nan = train[train["HomePlanet"]==3]
HomePlanet_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
59,3,1,2,33.0,0,0.0,0.0,0.0,0.0,0.0,True,4,1
113,3,0,2,39.0,0,0.0,2344.0,0.0,65.0,6898.0,False,0,0
186,3,1,0,24.0,0,0.0,0.0,0.0,0.0,0.0,True,3,0
225,3,0,2,18.0,0,313.0,1.0,691.0,283.0,0.0,False,5,1
234,3,1,0,54.0,0,0.0,0.0,0.0,0.0,0.0,True,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8515,3,0,2,25.0,0,1258.0,0.0,22.0,19.0,0.0,False,4,0
8613,3,0,0,53.0,0,0.0,4017.0,0.0,13.0,3147.0,False,4,1
8666,3,0,0,38.0,2,28.0,1208.0,973.0,207.0,0.0,True,5,1
8674,3,0,2,13.0,0,39.0,0.0,1085.0,24.0,0.0,False,5,0


In [36]:
le_CryoSleep.classes_

array([False, True, nan], dtype=object)

In [37]:
CryoSleep_Nan = train[train["CryoSleep"]==2]
CryoSleep_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
92,0,2,2,2.0,0,0.0,0.0,0.0,0.0,0.0,True,6,0
98,0,2,2,27.0,0,0.0,0.0,570.0,2.0,131.0,False,5,0
104,1,2,2,40.0,0,0.0,331.0,0.0,0.0,1687.0,False,1,0
111,2,2,2,26.0,0,0.0,0.0,0.0,0.0,0.0,True,5,0
152,0,2,2,58.0,0,0.0,985.0,0.0,5.0,0.0,True,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,1,2,0,44.0,0,0.0,0.0,0.0,0.0,0.0,True,2,0
8651,0,2,2,8.0,0,0.0,0.0,0.0,0.0,0.0,False,6,0
8664,0,2,2,32.0,0,0.0,0.0,0.0,0.0,0.0,True,6,1
8675,0,2,2,44.0,0,1030.0,1015.0,0.0,11.0,0.0,True,5,0


In [38]:
le_Destination.classes_

array(['55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e', nan], dtype=object)

In [39]:
Destination_Nan = train[train["Destination"]==3]
Destination_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
47,2,1,3,19.0,0,0.0,0.0,0.0,0.0,0.0,True,5,0
128,0,0,3,34.0,0,0.0,22.0,0.0,564.0,207.0,False,4,0
139,0,0,3,41.0,0,0.0,0.0,0.0,0.0,607.0,False,5,0
347,3,0,3,23.0,0,348.0,0.0,0.0,4.0,368.0,False,6,0
430,0,1,3,50.0,0,0.0,0.0,0.0,0.0,0.0,False,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8372,0,1,3,20.0,0,0.0,0.0,0.0,0.0,0.0,True,6,0
8551,2,1,3,41.0,0,0.0,0.0,0.0,0.0,0.0,True,5,1
8616,2,1,3,33.0,0,0.0,0.0,0.0,0.0,0.0,True,5,1
8621,1,0,3,41.0,1,0.0,7964.0,0.0,3238.0,5839.0,False,2,0


In [40]:
le_VIP.classes_

array([False, True, nan], dtype=object)

In [41]:
VIP_Nan = train[train["VIP"]==2]
VIP_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
38,0,0,0,15.0,2,0.0,492.0,48.0,20.0,401.0,False,5,1
102,0,0,2,0.0,2,0.0,0.0,0.0,0.0,0.0,True,6,1
145,2,1,2,35.0,2,0.0,0.0,0.0,0.0,0.0,True,5,0
228,2,1,0,14.0,2,0.0,0.0,0.0,0.0,0.0,True,5,1
582,1,0,2,37.0,2,27.0,654.0,10.0,881.0,9568.0,False,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,0,1,2,0.0,2,0.0,0.0,0.0,0.0,0.0,True,6,1
8512,0,0,1,16.0,2,0.0,0.0,761.0,0.0,0.0,False,5,0
8542,0,1,0,55.0,2,0.0,0.0,0.0,0.0,0.0,False,6,1
8630,1,1,2,52.0,2,0.0,0.0,0.0,0.0,0.0,True,1,0


In [42]:
le_Cabin1.classes_

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', nan], dtype=object)

In [43]:
Cabin1_Nan = train[train["Cabin1"]==8]
Cabin1_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
15,0,0,2,31.0,0,32.0,0.0,876.0,0.0,0.0,False,8,2
93,2,1,2,31.0,0,0.0,0.0,0.0,0.0,0.0,True,8,2
103,1,0,2,32.0,0,0.0,410.0,6.0,3929.0,764.0,False,8,2
222,2,0,2,37.0,0,637.0,0.0,0.0,92.0,319.0,False,8,2
227,2,1,2,43.0,0,0.0,0.0,0.0,0.0,0.0,True,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8209,1,0,0,53.0,0,0.0,1127.0,0.0,3939.0,0.0,False,8,2
8475,1,0,0,36.0,1,132.0,3479.0,0.0,3786.0,0.0,False,8,2
8485,1,1,0,25.0,0,0.0,0.0,0.0,0.0,0.0,True,8,2
8509,0,1,2,1.0,0,0.0,0.0,0.0,0.0,0.0,False,8,2


In [44]:
le_Cabin3.classes_

array(['P', 'S', nan], dtype=object)

In [45]:
Cabin3_Nan = train[train["Cabin3"]==2]
Cabin3_Nan

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
15,0,0,2,31.0,0,32.0,0.0,876.0,0.0,0.0,False,8,2
93,2,1,2,31.0,0,0.0,0.0,0.0,0.0,0.0,True,8,2
103,1,0,2,32.0,0,0.0,410.0,6.0,3929.0,764.0,False,8,2
222,2,0,2,37.0,0,637.0,0.0,0.0,92.0,319.0,False,8,2
227,2,1,2,43.0,0,0.0,0.0,0.0,0.0,0.0,True,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8209,1,0,0,53.0,0,0.0,1127.0,0.0,3939.0,0.0,False,8,2
8475,1,0,0,36.0,1,132.0,3479.0,0.0,3786.0,0.0,False,8,2
8485,1,1,0,25.0,0,0.0,0.0,0.0,0.0,0.0,True,8,2
8509,0,1,2,1.0,0,0.0,0.0,0.0,0.0,0.0,False,8,2


## 결측치 데이터 수정(연속형만)

In [21]:
# 나이 데이터는 없을 경우 삭제, 다른 비용관련 데이터는 0이므로 결측치로 나왔다고 가정하여 0으로 수정

In [22]:
train = train[train["Age"].notnull()]

In [23]:
train.loc[train['RoomService'] != train['RoomService'], 'RoomService'] = 0
train.loc[train['FoodCourt'] != train['FoodCourt'], 'FoodCourt'] = 0
train.loc[train['ShoppingMall'] != train['ShoppingMall'], 'ShoppingMall'] = 0
train.loc[train['Spa'] != train['Spa'], 'Spa'] = 0
train.loc[train['VRDeck'] != train['VRDeck'], 'VRDeck'] = 0

In [24]:
train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin2,Cabin3
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False,1,0,0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,True,5,0,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,0,0,1
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,0,0,1
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,True,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,1,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,False,0,98,0
8689,0,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,False,6,1499,1
8690,0,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,True,6,1500,1
8691,1,0,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,False,4,608,1


In [25]:
train.drop(columns="Cabin2",inplace=True)

C:\Users\신동혁\AppData\Local\Temp\ipykernel_22788\215921869.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(columns="Cabin2",inplace=True)


In [26]:
train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False,1,0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,True,5,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,0,1
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,0,1
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,True,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,1,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,False,0,0
8689,0,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,False,6,1
8690,0,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,True,6,1
8691,1,0,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,False,4,1


In [34]:
train["HomePlanet"].unique()

array([1, 0, 2, 3])

In [46]:
Nan_index = HomePlanet_Nan.index.append([CryoSleep_Nan.index, Destination_Nan.index, VIP_Nan.index, Cabin1_Nan.index,Cabin3_Nan.index])
Nan_index_N = Nan_index[Nan_index.duplicated(keep="first")]
Nan_index = Nan_index[~Nan_index.duplicated(keep="first")]


In [47]:
len(Nan_index), len(Nan_index_N)

(942, 239)

In [48]:
train_intact = train.drop(Nan_index)
train_intact

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False,1,0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,True,5,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,0,1
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,0,1
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,True,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,1,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,False,0,0
8689,0,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,False,6,1
8690,0,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,True,6,1
8691,1,0,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,False,4,1


# 결측치 예측을 위한 학습

In [49]:
print(le_HomePlanet.classes_)
print(le_CryoSleep.classes_)
print(le_Destination.classes_)
print(le_VIP.classes_)
print(le_Cabin1.classes_)
print(le_Cabin3.classes_)

['Earth' 'Europa' 'Mars' nan]
[False True nan]
['55 Cancri e' 'PSO J318.5-22' 'TRAPPIST-1e' nan]
[False True nan]
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'T' nan]
['P' 'S' nan]


학습을 위한 전처리는 마무리되었으므로, 학습 실행

## 모델 : 범주형( DecisionTreeClassifier, RandomForestClassifier )

In [50]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

tree = DecisionTreeClassifier(random_state=0)
forest = RandomForestClassifier(random_state=0)

params ={
    "max_depth":[3, 6, 9, 12, 15]
}

gs_tree = GridSearchCV(tree,
                      param_grid=params,
                      scoring="accuracy",
                      cv=4,
                      n_jobs=-1)
gs_forest = GridSearchCV(forest,
                      param_grid=params,
                      scoring="accuracy",
                      cv=4,
                      n_jobs=-1)

### HomePlanet

In [51]:
X_train = train_intact.drop(columns="HomePlanet")
y_train = train_intact["HomePlanet"]
gs_tree.fit(X_train, y_train)
gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [52]:
gs_tree.best_score_, gs_tree.best_params_

(0.9243264659270999, {'max_depth': 15})

In [53]:
gs_forest.best_score_, gs_forest.best_params_

(0.9399101954569467, {'max_depth': 15})

### CryoSleep

In [54]:
X_train = train_intact.drop(columns="CryoSleep")
y_train = train_intact["CryoSleep"]
gs_tree.fit(X_train, y_train)
gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [55]:
gs_tree.best_score_, gs_tree.best_params_

(0.9512678288431062, {'max_depth': 12})

In [56]:
gs_forest.best_score_, gs_forest.best_params_

(0.950607501320655, {'max_depth': 15})

### Destination

In [57]:
X_train = train_intact.drop(columns="Destination")
y_train = train_intact["Destination"]
gs_tree.fit(X_train, y_train)
gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [58]:
gs_tree.best_score_, gs_tree.best_params_

(0.692419440042261, {'max_depth': 3})

In [59]:
gs_forest.best_score_, gs_forest.best_params_

(0.6958531431590068, {'max_depth': 6})

### VIP

In [60]:
X_train = train_intact.drop(columns="VIP")
y_train = train_intact["VIP"]
gs_tree.fit(X_train, y_train)
gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [61]:
gs_tree.best_score_, gs_tree.best_params_

(0.9759640781827786, {'max_depth': 3})

In [62]:
gs_forest.best_score_, gs_forest.best_params_

(0.9759640781827786, {'max_depth': 3})

### Cabin1

In [63]:
X_train = train_intact.drop(columns="Cabin1")
y_train = train_intact["Cabin1"]
gs_tree.fit(X_train, y_train)
gs_forest.fit(X_train, y_train)

C:\anaconda\envs\da\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(
C:\anaconda\envs\da\lib\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(


GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [64]:
gs_tree.best_score_, gs_tree.best_params_

(0.6277073428420497, {'max_depth': 6})

In [65]:
gs_forest.best_score_, gs_forest.best_params_

(0.6405176967776016, {'max_depth': 9})

### Cabin3

In [66]:
X_train = train_intact.drop(columns="Cabin3")
y_train = train_intact["Cabin3"]
gs_tree.fit(X_train, y_train)
gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [67]:
gs_tree.best_score_, gs_tree.best_params_

(0.53486529318542, {'max_depth': 3})

In [68]:
gs_forest.best_score_, gs_forest.best_params_

(0.5450343370311674, {'max_depth': 3})

In [69]:
# 현재 HomePlanet, CryoSleep, VIP를 제외하고서는 정확도가 너무 낮게 나온다. 정확도가 높은 항목들만 일단 결측치를 채워 넣는다.

### 정확도 높은 항목들만 결측치 채우기(HomePlanet, CryoSleep, VIP)

In [70]:
X_train = train_intact.drop(columns="HomePlanet")
y_train = train_intact["HomePlanet"]

gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [71]:
a = HomePlanet_Nan.dropna()
a = a[a["CryoSleep"]!=2]
a = a[a["Destination"]!=3]
a = a[a["VIP"]!=2]
a

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
59,3,1,2,33.0,0,0.0,0.0,0.0,0.0,0.0,True,4,1
113,3,0,2,39.0,0,0.0,2344.0,0.0,65.0,6898.0,False,0,0
186,3,1,0,24.0,0,0.0,0.0,0.0,0.0,0.0,True,3,0
225,3,0,2,18.0,0,313.0,1.0,691.0,283.0,0.0,False,5,1
234,3,1,0,54.0,0,0.0,0.0,0.0,0.0,0.0,True,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,3,1,2,23.0,0,0.0,0.0,0.0,0.0,0.0,True,5,1
8515,3,0,2,25.0,0,1258.0,0.0,22.0,19.0,0.0,False,4,0
8613,3,0,0,53.0,0,0.0,4017.0,0.0,13.0,3147.0,False,4,1
8674,3,0,2,13.0,0,39.0,0.0,1085.0,24.0,0.0,False,5,0


In [72]:
a["HomePlanet"] = gs_forest.predict(a.drop(columns="HomePlanet"))
a

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
59,2,1,2,33.0,0,0.0,0.0,0.0,0.0,0.0,True,4,1
113,1,0,2,39.0,0,0.0,2344.0,0.0,65.0,6898.0,False,0,0
186,1,1,0,24.0,0,0.0,0.0,0.0,0.0,0.0,True,3,0
225,0,0,2,18.0,0,313.0,1.0,691.0,283.0,0.0,False,5,1
234,1,1,0,54.0,0,0.0,0.0,0.0,0.0,0.0,True,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,2,1,2,23.0,0,0.0,0.0,0.0,0.0,0.0,True,5,1
8515,2,0,2,25.0,0,1258.0,0.0,22.0,19.0,0.0,False,4,0
8613,1,0,0,53.0,0,0.0,4017.0,0.0,13.0,3147.0,False,4,1
8674,2,0,2,13.0,0,39.0,0.0,1085.0,24.0,0.0,False,5,0


In [73]:
indexes = a.index
for index in indexes:
    train.loc[index] = a.loc[index]

In [74]:
X_train = train_intact.drop(columns="CryoSleep")
y_train = train_intact["CryoSleep"]

gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [75]:
a = CryoSleep_Nan.dropna()
a = a[a["HomePlanet"]!=3]
a = a[a["Destination"]!=3]
a = a[a["VIP"]!=2]
a

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
92,0,2,2,2.0,0,0.0,0.0,0.0,0.0,0.0,True,6,0
98,0,2,2,27.0,0,0.0,0.0,570.0,2.0,131.0,False,5,0
104,1,2,2,40.0,0,0.0,331.0,0.0,0.0,1687.0,False,1,0
111,2,2,2,26.0,0,0.0,0.0,0.0,0.0,0.0,True,5,0
152,0,2,2,58.0,0,0.0,985.0,0.0,5.0,0.0,True,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,1,2,0,44.0,0,0.0,0.0,0.0,0.0,0.0,True,2,0
8651,0,2,2,8.0,0,0.0,0.0,0.0,0.0,0.0,False,6,0
8664,0,2,2,32.0,0,0.0,0.0,0.0,0.0,0.0,True,6,1
8675,0,2,2,44.0,0,1030.0,1015.0,0.0,11.0,0.0,True,5,0


In [76]:
a["CryoSleep"] = gs_forest.predict(a.drop(columns="CryoSleep"))
a

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
92,0,1,2,2.0,0,0.0,0.0,0.0,0.0,0.0,True,6,0
98,0,0,2,27.0,0,0.0,0.0,570.0,2.0,131.0,False,5,0
104,1,0,2,40.0,0,0.0,331.0,0.0,0.0,1687.0,False,1,0
111,2,1,2,26.0,0,0.0,0.0,0.0,0.0,0.0,True,5,0
152,0,0,2,58.0,0,0.0,985.0,0.0,5.0,0.0,True,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,1,1,0,44.0,0,0.0,0.0,0.0,0.0,0.0,True,2,0
8651,0,0,2,8.0,0,0.0,0.0,0.0,0.0,0.0,False,6,0
8664,0,1,2,32.0,0,0.0,0.0,0.0,0.0,0.0,True,6,1
8675,0,0,2,44.0,0,1030.0,1015.0,0.0,11.0,0.0,True,5,0


In [77]:
indexes = a.index
for index in indexes:
    train.loc[index] = a.loc[index]

In [78]:
X_train = train_intact.drop(columns="VIP")
y_train = train_intact["VIP"]

gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [79]:
a = VIP_Nan.dropna()
a = a[a["HomePlanet"]!=3]
a = a[a["Destination"]!=3]
a = a[a["CryoSleep"]!=2]
a

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
38,0,0,0,15.0,2,0.0,492.0,48.0,20.0,401.0,False,5,1
102,0,0,2,0.0,2,0.0,0.0,0.0,0.0,0.0,True,6,1
145,2,1,2,35.0,2,0.0,0.0,0.0,0.0,0.0,True,5,0
228,2,1,0,14.0,2,0.0,0.0,0.0,0.0,0.0,True,5,1
582,1,0,2,37.0,2,27.0,654.0,10.0,881.0,9568.0,False,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8430,1,1,2,33.0,2,0.0,0.0,0.0,0.0,0.0,True,1,1
8494,0,1,2,0.0,2,0.0,0.0,0.0,0.0,0.0,True,6,1
8512,0,0,1,16.0,2,0.0,0.0,761.0,0.0,0.0,False,5,0
8542,0,1,0,55.0,2,0.0,0.0,0.0,0.0,0.0,False,6,1


In [80]:
a["VIP"] = gs_forest.predict(a.drop(columns="VIP"))
a

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
38,0,0,0,15.0,0,0.0,492.0,48.0,20.0,401.0,False,5,1
102,0,0,2,0.0,0,0.0,0.0,0.0,0.0,0.0,True,6,1
145,2,1,2,35.0,0,0.0,0.0,0.0,0.0,0.0,True,5,0
228,2,1,0,14.0,0,0.0,0.0,0.0,0.0,0.0,True,5,1
582,1,0,2,37.0,0,27.0,654.0,10.0,881.0,9568.0,False,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8430,1,1,2,33.0,0,0.0,0.0,0.0,0.0,0.0,True,1,1
8494,0,1,2,0.0,0,0.0,0.0,0.0,0.0,0.0,True,6,1
8512,0,0,1,16.0,0,0.0,0.0,761.0,0.0,0.0,False,5,0
8542,0,1,0,55.0,0,0.0,0.0,0.0,0.0,0.0,False,6,1


In [81]:
indexes = a.index
for index in indexes:
    train.loc[index] = a.loc[index]

# 나머지 결측치 처리

In [82]:
# 결측치가 한 faature에만 존재하는 경우는 대충 결측치를 학습하여 넣어줬다.
# 남은 데이터들은 결측치가 여러군데 존재하는 경우로 학습이 어려운 데이터이므로 그냥 삭제한다.

In [88]:
train.drop(train[train["HomePlanet"]==3].index, inplace=True)

C:\Users\신동혁\AppData\Local\Temp\ipykernel_22788\1798854277.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(train[train["HomePlanet"]==3].index, inplace=True)


In [89]:
train.drop(train[train["CryoSleep"]==2].index, inplace=True)

C:\Users\신동혁\AppData\Local\Temp\ipykernel_22788\1547244750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(train[train["CryoSleep"]==2].index, inplace=True)


In [90]:
train.drop(train[train["Destination"]==3].index, inplace=True)

C:\Users\신동혁\AppData\Local\Temp\ipykernel_22788\167110041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(train[train["Destination"]==3].index, inplace=True)


In [91]:
train.drop(train[train["VIP"]==2].index, inplace=True)

C:\Users\신동혁\AppData\Local\Temp\ipykernel_22788\3191166109.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(train[train["VIP"]==2].index, inplace=True)


In [93]:
train

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1,Cabin3
0,1,0,2,39.0,0,0.0,0.0,0.0,0.0,0.0,False,1,0
1,0,0,2,24.0,0,109.0,9.0,25.0,549.0,44.0,True,5,1
2,1,0,2,58.0,1,43.0,3576.0,0.0,6715.0,49.0,False,0,1
3,1,0,2,33.0,0,0.0,1283.0,371.0,3329.0,193.0,False,0,1
4,0,0,2,16.0,0,303.0,70.0,151.0,565.0,2.0,True,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,1,0,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,False,0,0
8689,0,1,1,18.0,0,0.0,0.0,0.0,0.0,0.0,False,6,1
8690,0,0,2,26.0,0,0.0,0.0,1872.0,1.0,0.0,True,6,1
8691,1,0,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,False,4,1


# Transported 학습하기

In [95]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

tree = DecisionTreeClassifier(random_state=0)
forest = RandomForestClassifier(random_state=0)

params ={
    "max_depth":[3, 6, 9, 12, 15]
}

gs_tree = GridSearchCV(tree,
                      param_grid=params,
                      scoring="accuracy",
                      cv=4,
                      n_jobs=-1)
gs_forest = GridSearchCV(forest,
                      param_grid=params,
                      scoring="accuracy",
                      cv=4,
                      n_jobs=-1)

In [96]:
X_train = train.drop(columns="Transported")
y_train = train["Transported"]

In [97]:
gs_tree.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=DecisionTreeClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [98]:
gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [99]:
gs_tree.best_score_

0.7725852955309948

In [100]:
gs_forest.best_score_

0.7924074963959634

In [102]:
my_estimator = gs_forest.best_estimator_

# Test 데이터에 적용해보기

In [158]:
test = pd.read_csv("test.csv")

In [159]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [160]:
# 위에서 학습시켰던 형태로 변환

In [161]:
test[["Cabin1","Cabin2", "Cabin3"]] = test["Cabin"].str.split("/", expand=True)
test.drop(columns=["PassengerId","Name","Cabin","Cabin2"],inplace=True)
test.loc[test['RoomService'] != test['RoomService'], 'RoomService'] = 0
test.loc[test['FoodCourt'] != test['FoodCourt'], 'FoodCourt'] = 0
test.loc[test['ShoppingMall'] != test['ShoppingMall'], 'ShoppingMall'] = 0
test.loc[test['Spa'] != test['Spa'], 'Spa'] = 0
test.loc[test['VRDeck'] != test['VRDeck'], 'VRDeck'] = 0
le_HomePlanet = LabelEncoder()
test["HomePlanet"] = le_HomePlanet.fit_transform(test["HomePlanet"])

le_CryoSleep = LabelEncoder()
test["CryoSleep"] = le_CryoSleep.fit_transform(test["CryoSleep"])

le_Destination = LabelEncoder()
test["Destination"] = le_Destination.fit_transform(test["Destination"])

le_VIP = LabelEncoder()
test["VIP"] = le_VIP.fit_transform(test["VIP"])

le_Cabin1 = LabelEncoder()
test["Cabin1"] = le_Cabin1.fit_transform(test["Cabin1"])

le_Cabin3 = LabelEncoder()
test["Cabin3"] = le_Cabin3.fit_transform(test["Cabin3"])

test

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1,Cabin3
0,0,1,2,27.0,0,0.0,0.0,0.0,0.0,0.0,6,1
1,0,0,2,19.0,0,0.0,9.0,0.0,2823.0,0.0,5,1
2,1,1,0,31.0,0,0.0,0.0,0.0,0.0,0.0,2,1
3,1,0,2,38.0,0,0.0,6652.0,0.0,181.0,585.0,2,1
4,0,0,2,20.0,0,10.0,0.0,635.0,0.0,0.0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0,1,2,34.0,0,0.0,0.0,0.0,0.0,0.0,6,1
4273,0,0,2,42.0,0,0.0,847.0,17.0,10.0,144.0,8,2
4274,2,1,0,NaN,0,0.0,0.0,0.0,0.0,0.0,3,0
4275,1,0,3,NaN,0,0.0,2680.0,0.0,0.0,523.0,3,0


In [163]:
result = test[test["Age"].notnull()]["Age"]
result.mean()

28.65814620162446

In [164]:
test["Age"] = test["Age"].fillna(29.0)

In [165]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4277 non-null   int32  
 1   CryoSleep     4277 non-null   int32  
 2   Destination   4277 non-null   int32  
 3   Age           4277 non-null   float64
 4   VIP           4277 non-null   int32  
 5   RoomService   4277 non-null   float64
 6   FoodCourt     4277 non-null   float64
 7   ShoppingMall  4277 non-null   float64
 8   Spa           4277 non-null   float64
 9   VRDeck        4277 non-null   float64
 10  Cabin1        4277 non-null   int32  
 11  Cabin3        4277 non-null   int32  
dtypes: float64(6), int32(6)
memory usage: 300.9 KB


In [166]:
X_train = train_intact.drop(columns=["HomePlanet","Transported"])
y_train = train_intact["HomePlanet"]

gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [167]:
a = test[(test["HomePlanet"]==3)&(test["CryoSleep"]!=2)&(test["Destination"]!=3)&(test["VIP"]!=2)]


gs_forest.predict(a.drop(columns="HomePlanet"))

array([0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2,
       0, 1, 1, 2, 0, 1, 0, 2, 1, 0, 2, 1, 1, 0, 2, 0, 2, 2, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 2, 2, 2, 2, 0, 2, 1, 0, 1, 2, 0, 0, 0, 2,
       2, 1, 0, 1, 0, 0, 0, 2, 0, 1, 1, 0, 0, 2, 0, 2])

In [173]:
test.loc[a.index,"HomePlanet"]=gs_forest.predict(a.drop(columns="HomePlanet"))

In [178]:
result = test[test["HomePlanet"]==3].index
test.loc[result, "HomePlanet"]=0

In [179]:
test[test["HomePlanet"]==3]

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1,Cabin3


In [182]:
X_train = train_intact.drop(columns=["CryoSleep","Transported"])
y_train = train_intact["CryoSleep"]

gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [183]:
a = test[(test["CryoSleep"]==2)&(test["Destination"]!=3)&(test["VIP"]!=2)]


gs_forest.predict(a.drop(columns="CryoSleep"))

array([1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1])

In [184]:
test.loc[a.index,"CryoSleep"]=gs_forest.predict(a.drop(columns="CryoSleep"))

In [185]:
result = test[test["CryoSleep"]==2].index
test.loc[result, "CryoSleep"]=0

In [187]:
test[test["CryoSleep"]==2]

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1,Cabin3


In [188]:
X_train = train_intact.drop(columns=["Destination","Transported"])
y_train = train_intact["Destination"]

gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [189]:
a = test[(test["Destination"]==3)&(test["VIP"]!=2)]


gs_forest.predict(a.drop(columns="Destination"))

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2])

In [190]:
test.loc[a.index,"Destination"]=gs_forest.predict(a.drop(columns="Destination"))

In [191]:
result = test[test["Destination"]==3].index
test.loc[result, "Destination"]=0

In [192]:
test[test["Destination"]==3]

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1,Cabin3


In [193]:
X_train = train_intact.drop(columns=["VIP","Transported"])
y_train = train_intact["VIP"]

gs_forest.fit(X_train, y_train)

GridSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15]}, scoring='accuracy')

In [194]:
a = test[(test["VIP"]==2)]


gs_forest.predict(a.drop(columns="VIP"))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [195]:
test.loc[a.index,"VIP"]=gs_forest.predict(a.drop(columns="VIP"))

In [196]:
result = test[test["VIP"]==2].index
test.loc[result, "VIP"]=0

In [197]:
test[test["VIP"]==2]

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1,Cabin3


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1,Cabin3
count,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000,4277.000000
mean,0.680150,0.368483,1.511807,28.665420,0.017302,215.062427,428.592238,173.233107,295.895955,304.898293,4.438859,0.536123
std,0.811869,0.482450,0.804666,14.027471,0.130409,601.914503,1510.155974,554.991776,1104.872018,1235.991811,1.786765,0.543623
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000
50%,0.000000,0.000000,2.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,1.000000
75%,1.000000,1.000000,2.000000,37.000000,0.000000,48.000000,66.000000,27.000000,43.000000,31.000000,6.000000,1.000000
max,2.000000,1.000000,2.000000,79.000000,1.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000,8.000000,2.000000


In [201]:
my_estimator

RandomForestClassifier(max_depth=15, random_state=0)

In [203]:
answer = my_estimator.predict(test)

In [204]:
answer

array([ True, False,  True, ...,  True,  True, False])

In [206]:
passengerid = pd.read_csv("test.csv")["PassengerId"]
passengerid

0       0013_01
1       0018_01
2       0019_01
3       0021_01
4       0023_01
         ...   
4272    9266_02
4273    9269_01
4274    9271_01
4275    9273_01
4276    9277_01
Name: PassengerId, Length: 4277, dtype: object

In [208]:
final = pd.DataFrame({
    "PassengerId":passengerid,
    "Transported":answer
})
final

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [210]:
final.to_csv("second.csv", index=False)